# 4-CudaDataframe

This notebook demonstrates how to read and process a tabular datafile with the [cuDF](https://docs.rapids.ai/api/cudf/stable/) GPU dataframe library. The size of data is limited by the available GPU memory. cuDF provides a pandas-like API.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [ ]:
import os
import cudf
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [ ]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
print("Filename:", filename)

### Setup Benchmark

The ```file_format``` parameter is used for benchmarking different file formats. 
This cell below has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [ ]:
file_format = "csv"

In [ ]:
start = time.time()

### Read Data

In [ ]:
# read only specified columns and rows
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]
filters=[[("type_of_gene", "==", "protein-coding")]]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = cudf.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
    genes = genes[genes["type_of_gene"] == 'protein-coding']    
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = cudf.read_parquet(filename, columns=column_names, filters=filters)
else:
    print("invalid file format")
    
print("Filename:", filename)
    
genes = genes.rename(columns={"#tax_id": "tax_id"})

### Process Data

In [ ]:
groups = genes.groupby("tax_id").size().reset_index()
groups.columns = ["tax_id", "count"]
groups = groups.sort_values("count", ascending=False)

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [ ]:
groups[groups["tax_id"]  == "9606"]

#### Top 5 organisms with the most protein-coding genes

In [ ]:
groups.head()

In [ ]:
end = time.time()

In [ ]:
print(f"cuDF: {end - start:.1f} sec.")